In [1]:
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

In [2]:
cid = '5b16dbe9a3f144cf802ab7fe13b2c906'
secret = 'a19d0b35389e4d1dba98b2b9faf0ab52'
redirect_uri='https://localhost:8888/callback'
username = 'Meliodas'

In [3]:
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [4]:
# Getting playlist IDs from each of Spotify's playlists
playlists = sp.user_playlists('spotify')
spotify_playlist_ids = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        spotify_playlist_ids.append(playlist['uri'][-22:])
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None
spotify_playlist_ids[:20]

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX1lVhptIYRda',
 '37i9dQZF1DX10zKzsJ2jva',
 '37i9dQZF1DX4JAvHpjipBk',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWXRqgorJj26U',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DWXJfnUiYjUKT',
 '37i9dQZF1DXcRXFNfZr7Tp',
 '37i9dQZF1DX4o1oenSJRJd',
 '37i9dQZF1DXbTxeAdrVG2l',
 '37i9dQZF1DX4UtSsGT1Sbe',
 '37i9dQZF1DWTJ7xPn4vNaz',
 '37i9dQZF1DXaKIA8E7WcJj',
 '37i9dQZF1DWSV3Tk4GO2fq',
 '37i9dQZF1DWTwnEm1IYyoj',
 '37i9dQZF1DX2A29LI7xHn1']

In [5]:
len(spotify_playlist_ids)

1113

In [7]:
def getTrackFeatures(track_id):
#     print("started method")
    meta = sp.track(track_id)
#     print("assigned meta")
    features = sp.audio_features(track_id)
#     print("assigned features")
    
    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']
    genres = meta['album']['genres']

#     print("went thru meta")

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

#     print("went thru features")
    
    track = [track_id, name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, genres]
    return track

In [8]:
# Creating a function to get the first 50 tracks IDs from a playlist
def getTrackIDs(playlist_id):
    playlist = sp.user_playlist('spotify', playlist_id)
    for item in playlist['tracks']['items'][:50]:
        track = item['track']
        if track == None: continue
        ids.append(track['id'])
    return

In [9]:
%%time
# Gathering track ids
ids = []
for x in spotify_playlist_ids[:200]:
    getTrackIDs(x)
ids[:5]

CPU times: total: 2.73 s
Wall time: 1min 42s


['23RoR84KodL5HWvUTneQ1w',
 '741UUVE2kuITl0c6zuqqbO',
 '2IGMVunIBsBLtEQyoI1Mu7',
 '7x9aauaA9cu6tyfpHnqDLo',
 '2i8f4VnnBjy0yDqH2C452a']

In [10]:
len(ids)

9636

In [ ]:
%%time
# loop over track ids to get audio features for each track
tracks = []
for i in range(5):
    print(ids[i])
    try:  
        track = getTrackFeatures(ids[i])
        tracks.append(track)
    except:
        pass

# create dataset
df = pd.DataFrame(tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'genres'])
df.head()

23RoR84KodL5HWvUTneQ1w


In [ ]:
%%time
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id, limit = 100)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

results = get_playlist_tracks("Meliodas", "https://open.spotify.com/playlist/7lMeQW9qfnRn13H7xojaBs?si=9318b46daa774c01")

# results = sp.current_user_saved_tracks(limit=50)
liked_songs = []
# for item in results:
#     print(item)
#     print("\n")
for item in results:
    track = getTrackFeatures(item['track']['id'])
    liked_songs.append(track)

In [ ]:
df_fav = pd.DataFrame(liked_songs, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'genres'])
df_fav.head()

In [ ]:
df_fav.info()

In [ ]:
df_fav["favorite"] = 1
df_fav.head()

In [ ]:
df["favorite"] = 0
df.head()

In [ ]:
frames = [df, df_fav]
data = pd.concat(frames)
data.head()

In [ ]:
data.info()
data.to_csv("comb_data.csv")